In [1]:
import pandas as pd
import os
import funzioni as fx
import gc

max_df = 5000

lista_file = os.listdir("C:/Users/932197/OneDrive/Python/Dati_borsa/Dati_calcolati_scalati_approssimati_lagged")
tot_file = len(lista_file)
tickers = fx.tickers_de_giro("../DeGiro con formule.xlsx")

vero = [pd.DataFrame() for _ in range(max_df)]
falso = [pd.DataFrame() for _ in range(max_df)]
i = 0
for f in lista_file:
    nome_file, estensione = os.path.splitext(f)   
    categoria = tickers.loc[tickers["Ticker"] == nome_file, "Categoria"].iloc[0]
    print(f"\r{i} di {tot_file}) Caricamento ticker {nome_file}                                        ", end=' ', flush=True)
    df = pd.read_parquet(f"C:/Users/932197/OneDrive/Python/Dati_borsa/Dati_calcolati_scalati_approssimati_lagged/{f}")
    if (len(df) > 0) and (categoria != "D"):
        print(f"\r{i} di {tot_file}) Def.parametri e target ticker {nome_file}                            ", end=' ', flush=True)
        df = df.loc[df["Date"] >= "2013-01-01", :]
        df = fx.definizione_parametri_e_target(df, n_barre_passate=10)
        vero[i] = df.loc[df["Target"] == 1, :]
        falso[i] = (df.loc[df["Target"] == 0, :]).sample(n=(len(vero[i])) * 5)
        del(df)
#        lista_df[i_df]["Ticker"] = nome_file
#        lista_df[i_df]["Categoria"] = tickers.loc[tickers["Ticker"] == nome_file, "Categoria"].iloc[0]
        gc.collect()
    i += 1
    
print(f"\r{i} di {tot_file}) Concatenamento dataframe                                             ", end=' ', flush=True)
df_tot = pd.concat([vero[x] for x in range(max_df)] + [falso[x] for x in range(max_df)], axis=0)
            
print(f"\rSalvataggio su file totale                                                               ", end=' ', flush=True)
df_tot.to_parquet(f"C:/Users/932197/Documents/dati_locali/Dati_addestramento.parquet") 

del(df_tot)
del(vero)
del(falso)
gc.collect()

print("\nFine.")

Salvataggio su file totale                                                                                                                                                                                                                                                                                                                                                                                                                               
Fine.
